In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as m
from torchsummary import summary

print(f'PyTorch version: {torch.__version__}')
print("GPU found :)" if torch.cuda.is_available() else "No GPU :(")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

PyTorch version: 1.7.1
GPU found :)


In [2]:
EMBEDDING_SIZE = 300

In [3]:
# retrieve pretrained model for features extraction

resnet = m.resnet18(pretrained=True).to(device)
resnet

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/jchataigner/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
# Keep only the feature extraction layers of the model
features = nn.Sequential(*(list(resnet.children())[:-2]))
summary(features, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

## Build LSTM 

In [ ]:
class LSTMCaptioning(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        self.lstm = nn.LSTM(input_size, hidden_size)
        
    def forward(self, inputs):
         
        # Get hidden states for each t (out) , and latest one (h = (ht, ct))
        out, ht = self.lstm(inputs.view(len(inputs), 1, -1))
        
        # Compute probability distribution over all words for this t
        pt = F.softmax(ht, dim=1)
                           
        return out, ht, pt
        

## Train model

In [ ]:
input_size = (3, 224, 224)
hideen_size = ...

model = LSTMCaptioning(input_size, hidden_size)

num_epoch = 10
loss_function = nn.NLLLoss()

for epoch in range(num_epoch):
    for image, caption in dataset:
        
        # Reset grad
        model.zero_grad()
        
        # Get the input image embedding 
        embedding = features(image)
        targets = ...
        
        # Forward pass
        out, hidden, probs = model(embedding)
        
        # Compute loss and backprop
        loss = loss_function(out, targets)
        loss.backward()
        optimizer.step()
        

In [ ]:
# Save model 
torch.save(model.state_dict(), './models/')